In [3]:
# Dependencies
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import time
import re

In [4]:
# Create a browser object for use by all the scrapes.
# Need to do "brew install geckodriver" to use Firefox on Mac.
# Chrome doesn't correctly show the counters on Mac.
def init_browser():
    return Browser("firefox")

### Scrape https://map.webex.com/ and collect the counters for Jan, Feb, and Mar.


In [82]:
def scrape_webex():
    browser = init_browser()

    # Visit https://map.webex.com/
    url = "https://map.webex.com/"
    browser.visit(url)

    # Give time for dynamic content to load
    time.sleep(5)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # Get the title
    #scrape_title = soup.find(class_='content_title')
    #title = scrape_title.find_all('a')[0].text   
    
    #iframe_src = soup.select_one("#meetingMap").attrs["src"]

    month_ids = ['last2Month', 'lastMonth', 'nowMonth']
    months = []
    hosts_commas = []
    hosts = []
    participants_commas = []
    participants = []
    countries = []
    meetings_commas = []
    meetings = []
    minutes = []

    for month_id in month_ids:
        with browser.get_iframe('meetingMap') as iframe:
            iframe.click_link_by_id(month_id)
            time.sleep(5)
            iframe_html = iframe.html
            iframe_soup = bs(iframe_html, "html.parser")
            months.append(iframe_soup.find('span', id=month_id).get_text())
            #print(f'iframe_soup: {iframe_soup}')
            #hosts.append(iframe_soup.find('span', id='hostData'))
            hosts_commas.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='hostData'))).group(1))
            #hosts = ''.join(map(str, hosts))
            #hosts = hosts.replace(",", "")

            participants_commas.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='participantData'))).group(1))
            countries.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='countryData'))).group(1))
            meetings_commas.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='meetingData'))).group(1))
            
            mins = []
            num = 11
            while num > 0:
                a = str(iframe_soup.find('div', id='num' + str(num)))
                b = re.search(r'<span>(\d*)', a).group(1)             
                mins.append(b)
                num = num - 1
            mins_string = ''.join(map(str, mins))
            minutes.append(mins_string)
           
        
                
    hosts =[s.replace(',', '') for s in hosts_commas]
    participants = [s.replace(',', '') for s in participants_commas]
    meetings = [s.replace(',', '') for s in meetings_commas]
    
    webex = {
        "months": months,
        "hosts": hosts,
        "participants": participants,
        "countries": countries,
        "meetings": meetings,
        "minutes": minutes
    }
    

    # Close the browser after scraping
    browser.quit()

    # Return results
    return webex

In [83]:
webex = scrape_webex()
print(f'months: {webex["months"]}')
print(f'hosts: {webex["hosts"]}')
print(f'participants: {webex["participants"]}')
print(f'countries: {webex["countries"]}')
print(f'meetings: {webex["meetings"]}')
print(f'minutes: {webex["minutes"]}')

months: ['January 2020', 'February 2020', 'Current']
hosts: ['24504686', '24420815', '26000472']
participants: ['152398517', '161764828', '242339861']
countries: ['223', '224', '221']
meetings: ['36892704', '37994030', '52597353']
minutes: ['6310089577', '6787001490', '9804112710']
